##### Import package

In [4]:
from google.colab import drive
import os

from bs4 import BeautifulSoup
import requests, re, os, sys
from urllib.request import urlopen
from glob import glob
import pandas as pd

##### Using colab to run

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


##### Bible url

In [7]:
FR_URL = "https://www.jw.org/fr/biblioth%C3%A8que/bible/bi12/livres/"
MOS_URL = "https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/"

##### Env initialization

In [8]:
def get_books(lang_url):
    url = requests.get(lang_url)
    page =BeautifulSoup(url.text, 'lxml')
    books = page.find('select', attrs={'id':'Book'}).text.split('\n')[1:]
    
    for i in range(len(books)):
        if(len(books[i].split()) > 1):
            hyphen_join = books[i].split()
            books[i] = '-'.join(hyphen_join)
    
    return books

#### Downloading on jw300

In [72]:
FR_books = get_books(FR_URL)
FR_books.remove('')

MOS_books = get_books(MOS_URL)
MOS_books.remove('')


In [73]:
FR_books = FR_books[-len(MOS_books):]
books = pd.DataFrame([FR_books,MOS_books])
books

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
0,Matthieu,Marc,Luc,Jean,Actes,Romains,1-Corinthiens,2-Corinthiens,Galates,Éphésiens,Philippiens,Colossiens,1-Thessaloniciens,2-Thessaloniciens,1-Timothée,2-Timothée,Tite,Philémon,Hébreux,Jacques,1-Pierre,2-Pierre,1-Jean,2-Jean,3-Jean,Jude,Révélation
1,Matiye,Mark,Luk,Zã,Tʋʋma,Rom-dãmbã,1-Korẽnt-rãmb,2-Korẽnt-rãmb,Galat-rãmbã,Efɛɛz-rãmbã,Filip-rãmbã,Kolos-rãmbã,1-Tesalonik-rãmbã,2-Tesalonik-rãmbã,1-Tɩmote,2-Tɩmote,Tɩt,Filemo,Ebre-rãmbã,Zak,1-Pɩyɛɛr,2-Pɩyɛɛr,1-Zã,2-Zã,3-Zã,Ziid,Vẽnegre


#### Building the corpus

In [74]:
path="/content/drive/My Drive/Colab Notebooks/TraMoore/data_test/"
def write_book_to_file(sub_url, book,lang):
    for i in range(len(book)):
        os.makedirs(path+lang+book[i],exist_ok=True)
        address = sub_url + book[i]
        print(address)
        url = requests.get(address)
        page = BeautifulSoup(url.text, 'lxml')
        chapters = page.find('div', attrs={'class': 'chapters'}).text.split('\n')[1:]
        chapters.remove('')
        ## Get Chapters for Each book
        for ch in chapters:
            url1 = requests.get(sub_url + book[i] +'/' + ch)
            print(sub_url + book[i] +'/' + ch)
            page1 = BeautifulSoup(url1.text,'lxml')
            ch1 = page1.find('div',attrs={'id': "bibleText"})
            tt = [verses.text.replace(u'\xa0', u' ').replace('\n',' ') for verses in ch1.find_all('span',attrs={'class':'verse'})]
            chapter = open(path+lang+book[i]+"/"+str(ch) + ".txt", 'w')
            for item in tt:
                chapter.write("{}\n".format(item))

##### Downloads all chap and versé

In [ ]:
write_book_to_file(MOS_URL, MOS_book,"MOS/")
write_book_to_file(FR_URL, FR_book,"FR/")

In [77]:
# Initialise pour tester
MOS_books_test = MOS_books[0:2]
FR_books_test = FR_books[0:2]

In [80]:
write_book_to_file(MOS_URL, MOS_books_test,"MOS/")
write_book_to_file(FR_URL, FR_books_test,"FR/")

https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/1
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/2
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/3
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/4
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/5
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/6
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/7
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/8
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/9
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/10
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/11
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/bi12/S%C9%9Bb/Matiye/12
https://www.jw.org/mos/d-s%E1%BA%BDn-yiisi/biible/

##### Merge books

In [81]:
def merge_books(lang, books):
    file_lang = []
    for bk in books:
        file_lang.append((glob(path+lang+"/" + bk + "/*.txt")))

    
    with open(path+lang+"/All.txt","wb") as write_file:
        for f in file_lang:
            for i in f:
                with open(i,'rb') as r:
                    write_file.write(r.read())

In [82]:
merge_books("MOS",MOS_books_test)
merge_books("FR",FR_books_test)

##### Creating a parallel Corpus

In [88]:
mos = pd.read_csv(path+'MOS/All.txt',delimiter="\n",header=None,index_col=None)
mos.columns = ["MOSSI"]

fr = pd.read_csv(path+'FR/All.txt',delimiter="\n",header=None,index_col=None)
fr.columns = ["French"]

data = pd.concat([fr,mos],axis=1)
print(data.shape)
print(data.head(10))

data.to_csv(path+"fr_mos.csv",index=False)

(1749, 2)
                                              French                                              MOSSI
0  1  Le livre de l’histoire*+ de Jésus Christ, f...  1  Seb-kãngã gomda a Zeezi Kirist* vɩɩmã* sẽn ...
1   2  Abraham devint père d’*Isaac+ ;Isaac devin...   2  A Abrahaam n dog a Izaake,t’a Izaak rog a ...
2   3  Juda devint père de Pérets+ et de Zérah qu...   3  A Zida roga a Perɛɛz ne a Zeera, tɩ b ma y...
3   4  Ram devint père d’Amminadab ;Amminadab dev...   4  t’a Ram rog a Aminadaabe,t’a Aminadaab rog...
4   5  Salmôn devint père de Boaz qu’il eut de Ra...   5  t’a Salmo ne a pag a Rahaab rog a Boaaze,t...
5   6  Jessé devint père de David+ le roi+. David...   6  A Zese roga a Davɩɩd sẽn yɩ rĩmã,t’a Davɩɩ...
6   7  Salomon devint père de Rehabam*+ ;Rehabam ...   7  A Salomo roga a Roboam,t’a Roboam rog a Ab...
7   8  Asa devint père de Yehoshaphat*+ ;Yehoshap...   8  t’a Aaza rog a Zozafate,t’a Zozafat rog a ...
8   9  Ouzziya devint père de Yotham ;Yotham+ dev...  